In [1]:
import matplotlib.pyplot as plt
import numpy as np
import statistics as stats
import math
import random
import array

from time import time

#metrics
from sklearn import metrics
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import classification_report

#model_selection
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

#resampling
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek, SMOTEENN

#clasificadores
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import RUSBoostClassifier

#DEAP library for evolutionary algorithms
from deap import base
from deap import creator
from deap import tools

#datasets
from imblearn.datasets import fetch_datasets

In [ ]:
from collections import Counter

import numpy as np
from sklearn.base import is_regressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble.forest import BaseForest
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from sklearn.tree.tree import BaseDecisionTree
from sklearn.utils import check_random_state
from sklearn.utils import check_X_y
#from sklearn.utils import shuffle

In [2]:
#classifies sample x to the class which center is closer to
def classify(x,centers):
    dist_majcenter = euclidean(x,centers[:len(x)])
    dist_mincenter = euclidean(x,centers[len(x):])
    return np.argmin([dist_majcenter,dist_mincenter])

# function to generate the initial random population from the training set
def load_individuals(X,y,maj_class,min_class,creator,n):
    """

    """
    maj_samples = X[y == maj_class]
    min_samples = X[y == min_class]
    individuals = []
    for i in range(n):
        random_maj = maj_samples[random.randint(0,maj_samples.shape[0]-1)]
        random_min = min_samples[random.randint(0,min_samples.shape[0]-1)]
        individual = np.asarray(np.concatenate((random_maj,random_min)))

        individual = creator(individual)
        individuals.append(individual)
    return individuals

# returns the euclidean distance between two points
def euclidean(v1, v2):
    return sum((p-q)**2 for p, q in zip(v1, v2)) ** .5

#returns the sum of the distances from each sample in X_train to the closest center
#we are interested in minimizing this sum of distances
def evaluate(X,individual):
    D = X.shape[1]
    S = 0
    for x in X:
        dist = dist_to_closest_center(x,individual[:D],individual[D:])
        S += dist

    return S,

#computes the euclidean distance for both centers and returns the shortest one
def dist_to_closest_center(x,maj_center,min_center):
    dist_majcenter = euclidean(x,maj_center)
    dist_mincenter = euclidean(x,min_center)
    return min(dist_majcenter,dist_mincenter)

def unify_training_set(X,y,maj_class,min_class,boundary_points,nbp_us):
    new_majorityclass_training = np.vstack((boundary_points,nbp_us))
    print("Conjunto de entrenamiento original de tamaño: {}".format(X.shape[0]))
    n_may,n_min = sum(y == maj_class),sum(y == min_class)
    print("De los cuales:\n \t nº de ejemplos clase MAYORITARIA: {}\n \t nº de ejemplos clase MINORITARIA: {}"
          .format(n_may,n_min))
#         print("IR = {}".format(n_may/n_min))
    minority_samples = X[y==min_class]

    X_US = np.vstack((new_majorityclass_training,minority_samples))
    y_US = np.hstack((np.full(new_majorityclass_training.shape[0],maj_class),
                      np.full(minority_samples.shape[0],min_class)))
    print("nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: {}"
          .format(new_majorityclass_training.shape[0]))
    print("Conjunto de entrenamiento actual de tamaño: {}".format(new_majorityclass_training.shape[0]+n_min))
    return X_US,y_US

def rus(nbp,p=0.4):
    RUSsize = int(nbp.shape[0]*p)
    #RUS PURO
    indices = np.random.randint(nbp.shape[0], size=RUSsize)
    nbp_us = nbp[indices]
#         #RUS ponderado por el cluster stability de cada ejemplo
#         C_non_boundary = np.array(cluster_stabilities)[np.array(cluster_stabilities)>self.alpha]
#         indices = np.random.choice(np.arange(non_boundary_points.shape[0]),size=RUSsize,
#                                    p = C_non_boundary/sum(C_non_boundary))
#         nbp_us = non_boundary_points[indices]
    return nbp_us

In [28]:
class DEClustering(object):
    """
    DOCUMENTARRRRR
    """
    def __init__(self, CR=0.6,F=0.5,POP_SIZE=10,NGEN=100):
        
        self.CR = CR
        self.F = F
        self.POP_SIZE = POP_SIZE
        self.NGEN = NGEN
        
#         creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
#         creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMin)
#         self.toolbox = base.Toolbox()
#         self.toolbox.register("select", tools.selRandom, k=3)
        
        
    def fit(self, X_train, y_train,maj_class,min_class):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMin)
        self.toolbox = base.Toolbox()
        self.toolbox.register("select", tools.selRandom, k=3)
#         self.toolbox.unregister("population")
#         self.toolbox.unregister("evaluate")
        self.toolbox.register("population",load_individuals, X_train, y_train, maj_class, min_class,
                              creator.Individual)
        self.toolbox.register("evaluate", evaluate, X_train)
        
        NDIM = X_train.shape[1]
        
        pop = self.toolbox.population(n=self.POP_SIZE);
        hof = tools.HallOfFame(1)
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean)
        stats.register("std", np.std)
        stats.register("min", np.min)
        stats.register("max", np.max)

        logbook = tools.Logbook()
        logbook.header = "gen", "evals", "std", "min", "avg", "max"

        # Evaluate the individuals
        fitnesses = self.toolbox.map(self.toolbox.evaluate, pop)
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit

        record = stats.compile(pop)
        logbook.record(gen=0, evals=len(pop), **record)
#         print(logbook.stream)
        
        last_fitness = float('inf')
        for g in range(1, self.NGEN):
            for k, agent in enumerate(pop):
                a,b,c = self.toolbox.select(pop)
                y = self.toolbox.clone(agent)
                index = random.randrange(NDIM)
                for i, value in enumerate(agent):
                    if i == index or random.random() < self.CR:
                        y[i] = a[i] + self.F*(b[i]-c[i])
                y.fitness.values = self.toolbox.evaluate(y)
                if y.fitness > agent.fitness:
                    pop[k] = y
                #print(pop[k].fitness)
                
#                 if abs(last_fitness-agent.fitness.values[0])<1:
#                     g = NGEN
#                 last_fitness = agent.fitness.values[0]
            hof.update(pop)
            record = stats.compile(pop)
            logbook.record(gen=g, evals=len(pop), **record)
#             print(logbook.stream)

#         print("Best individual is ", hof[0], hof[0].fitness.values[0])
#         return hof[0]
        self.best_ind = hof[0]
        self.best_fitness = hof[0].fitness.values[0]
#         print(self.best_ind)
#         print(self.best_fitness)
        return self

In [4]:
class DECLUndersampling(object):
    def __init__(self, H=6, alpha=0.8, CR=None, F=None, POP_SIZE=None, NGEN=None):
        self.H = H
        self.alpha = alpha
         
        self.decl = DEClustering(CR, F, POP_SIZE, NGEN)
#         super(DECLUndersampling, self).__init__(X=X_train, y=y_train)
    
    
    def clustering_centers(self,X,y,maj_class,min_class):
        clustering_centers = []
        for i in range(int(self.H)):
            self.decl.fit(X,y,maj_class,min_class)
            centers = self.decl.best_ind
            clustering_centers.append(centers)
        return clustering_centers
    
    def cluster_stabilities(self,majority_samples,maj_class,min_class,clustering_centers):
        classes = [maj_class,min_class]
    
        cluster_stabilities = []
        for sample in majority_samples:
            S = 0
            for clustering in clustering_centers:
                c = classes[classify(sample,clustering)]
                if c==maj_class:
                    S += 1
            cluster_stabilities.append(S/self.H)
        return cluster_stabilities

    def undersample(self,X,y):
        
        classes,maj_class,min_class = convert_classes(y)
        
        #compute H clustering processes
        cl_c = self.clustering_centers(X,y,maj_class,min_class)
        
        #compute cluster stability for each majority sample
        majority_samples = X[y==maj_class]
        cl_st = self.cluster_stabilities(majority_samples,maj_class,min_class,cl_c)
            
        #compute boundary and non-boundary samples
        boundary_points = majority_samples[np.array(cl_st)<=self.alpha]
        non_boundary_points = majority_samples[np.array(cl_st)>self.alpha]
        
        #undersample non-boundary samples
        nbp_us = rus(non_boundary_points)
        
        #build undersampled training set
        X_US, y_US = unify_training_set(X,y,maj_class,min_class,boundary_points,nbp_us)

        return X_US, y_US

In [23]:
def Gmean(clf,X_test,y_test):
    y_pred = clf.predict(X_test)
    gmean = geometric_mean_score(y_test, y_pred)
    #print("Gmean:",gmean)
    return gmean
    
def compute_fitness(p,X_before_SMOTE,y_before_SMOTE,maj_class,min_class,synthetic_samples, individual):
    selected_syn = []
    for i, value in enumerate(individual):
        if individual[i]>0:
            selected_syn.append(synthetic_samples[i-1])
    if len(selected_syn)>0:
        selected_syn = np.array(selected_syn)
        X = np.vstack((X_before_SMOTE,selected_syn))
        y = np.hstack((y_before_SMOTE,np.full(selected_syn.shape[0],min_class)))
        #realizar splitTest??????????????''
        Xtr, Xtst, ytr, ytst = train_test_split(X, y, test_size=0.3)
        #dt = trainDT(Xtr,ytr,weights)
        dt = trainDT(Xtr,ytr)
        #test??????????????????????????
        #G = Gmean(dt,X_before_SMOTE,y_before_SMOTE)
        G = Gmean(dt,Xtst,ytst)
        n_minority = len(individual)+sum(individual)
        n_majority = X_before_SMOTE[y_before_SMOTE==maj_class].shape[0]
        f = G - abs(1-(n_minority/n_majority*p))
#         print("f: ",f)
        return G,
#         return f,
    else:
        return 0,

def trainDT(X_train,y_train,w=None):
    clf = DecisionTreeClassifier()
    clf = clf.fit(X_train,y_train,sample_weight=w)
    return clf

In [34]:
class DESMOTE(object):
    """
    DOCUMENTARRRRR
    """
    def __init__(self, CR, F ,POP_SIZE, NGEN, p=0.2):
        self.CR = CR
        self.F = F
        self.POP_SIZE = POP_SIZE
        self.NGEN = NGEN
        self.p = p
        
#         creator.create("FitnessMax", base.Fitness, weights=(1.0,))
#         creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMax, clf=None)
#         self.toolbox = base.Toolbox()
#         self.toolbox.register("attr_int", random.randint, 0, 1)

#         self.toolbox.register("select", tools.selRandom, k=3)
    
    
    def fit(self, X, y, maj_class, min_class, syn):
        
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMax, clf=None)
        self.toolbox = base.Toolbox()
        self.toolbox.register("attr_int", random.randint, 0, 1)

        self.toolbox.register("select", tools.selRandom, k=3)
        NDIM = syn.shape[0]
        
        self.toolbox.register("individual", tools.initRepeat, creator.Individual, self.toolbox.attr_int, NDIM)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)     
        self.toolbox.register("evaluate", compute_fitness, self.p, X, y, maj_class, min_class, syn)    
        
        pop = self.toolbox.population(n=self.POP_SIZE);
        hof = tools.HallOfFame(1)
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean)
        stats.register("std", np.std)
        stats.register("min", np.min)
        stats.register("max", np.max)

        logbook = tools.Logbook()
        logbook.header = "gen", "evals", "std", "min", "avg", "max"
        #print(pop)
        # Evaluate the individuals
        fitnesses = self.toolbox.map(self.toolbox.evaluate, pop)
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit

        record = stats.compile(pop)
        logbook.record(gen=0, evals=len(pop), **record)
        #print(logbook.stream)

        for g in range(1, self.NGEN):
            for k, agent in enumerate(pop):
                a,b,c = self.toolbox.select(pop)
                #we adopt a self-adaptative operator
                l = math.exp(1-(self.NGEN/(self.NGEN+1-g)))
                F_g = self.F*(2**l)
                d = self.toolbox.clone(agent) #donor vector
                sig_d = self.toolbox.clone(agent)
                y = self.toolbox.clone(agent)
                index = random.randrange(NDIM)
                for i, value in enumerate(agent):
                    d[i] = a[i] + F_g*(b[i]-c[i]) #donor vector
                    #the mutated donor is mapped to binary space by a sigmoid function with displacement
                    sig_d[i] = round(1/(1+math.exp(-(d[i]))))
                    if i == index or random.random() < self.CR:
#                         y[i] = a[i] + F*(b[i]-c[i])
                        y[i] = sig_d[i]
                y.fitness.values = self.toolbox.evaluate(y)
                if y.fitness > agent.fitness:
                    pop[k] = y
                #print(pop[k].fitness)
            hof.update(pop)
            record = stats.compile(pop)
            logbook.record(gen=g, evals=len(pop), **record)
#             print(logbook.stream)

#         print("Best individual is ", hof[0], hof[0].fitness.values[0])
#         return hof[0]
        self.best_ind = hof[0]
        self.best_fitness = hof[0].fitness.values[0]
#         print(self.best_ind)
#         print(self.best_fitness)
        return self

In [35]:
class DERSBoost(AdaBoostClassifier):
    """Implementation of DERSBoost.
    DERSBoost introduces data sampling into the AdaBoost algorithm by both
    undersampling the majority class guided by a Differential Evolutionary algorithm, and
    oversampling the minority class using also a DE-guided SMOTE procedure on each boosting iteration [1].
    This implementation inherits methods from the scikit-learn 
    AdaBoostClassifier class, only modifying the `fit` method.
    Parameters
    ----------
    n_samples : int, optional (default=100)
        Number of new synthetic samples per boosting step.
    k_neighbors : int, optional (default=5)
        Number of nearest neighbors.
    base_estimator : object, optional (default=DecisionTreeClassifier)
        The base estimator from which the boosted ensemble is built.
        Support for sample weighting is required, as well as proper `classes_`
        and `n_classes_` attributes.
    n_estimators : int, optional (default=50)
        The maximum number of estimators at which boosting is terminated.
        In case of perfect fit, the learning procedure is stopped early.
    learning_rate : float, optional (default=1.)
        Learning rate shrinks the contribution of each classifier by
        ``learning_rate``. There is a trade-off between ``learning_rate`` and
        ``n_estimators``.
    algorithm : {'SAMME', 'SAMME.R'}, optional (default='SAMME.R')
        If 'SAMME.R' then use the SAMME.R real boosting algorithm.
        ``base_estimator`` must support calculation of class probabilities.
        If 'SAMME' then use the SAMME discrete boosting algorithm.
        The SAMME.R algorithm typically converges faster than SAMME,
        achieving a lower test error with fewer boosting iterations.
    random_state : int or None, optional (default=None)
        If int, random_state is the seed used by the random number generator.
        If None, the random number generator is the RandomState instance used
        by np.random.
    References
    ----------
    .. [1] N. V. Chawla, A. Lazarevic, L. O. Hall, and K. W. Bowyer.
           "SMOTEBoost: Improving Prediction of the Minority Class in
           Boosting." European Conference on Principles of Data Mining and
           Knowledge Discovery (PKDD), 2003.
    """

    def __init__(self,
                 k_neighbors=5,
                 base_estimator=None,
                 n_estimators=50,
                 learning_rate=1.,
                 algorithm='SAMME.R',
                 random_state=None,
                 H=6,
                 CR=0.6,
                 F=0.5,
                 POP_SIZE=10,
                 NGEN=100,
                 us_alpha=0.8
                ):
        
        self.smote_kneighbors = k_neighbors
        self.algorithm = algorithm
#         self.maj_class = maj_class
#         self.min_class = min_class
        self.H = H
        self.CR = CR
        self.F = F
        self.POP_SIZE = POP_SIZE
        self.NGEN = NGEN
        self.us_alpha = us_alpha
        
        self.desmote = DESMOTE(self.CR,self.F,self.POP_SIZE,self.NGEN)

        super(DERSBoost, self).__init__(
            base_estimator=base_estimator,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            random_state=random_state)
        
    def fit(self, X, y, sample_weight=None, minority_target=None):
        """Build a boosted classifier/regressor from the training set (X, y),
        performing SMOTE during each boosting step.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape = [n_samples, n_features]
            The training input samples. Sparse matrix can be CSC, CSR, COO,
            DOK, or LIL. COO, DOK, and LIL are converted to CSR. The dtype is
            forced to DTYPE from tree._tree if the base classifier of this
            ensemble weighted boosting classifier is a tree or forest.
        y : array-like of shape = [n_samples]
            The target values (class labels in classification, real numbers in
            regression).
        sample_weight : array-like of shape = [n_samples], optional
            Sample weights. If None, the sample weights are initialized to
            1 / n_samples.
        minority_target : int
            Minority class label.
        Returns
        -------
        self : object
            Returns self.
        Notes
        -----
        Based on the scikit-learn v0.18 AdaBoostClassifier and
        BaseWeightBoosting `fit` methods.
        """
        # Check that algorithm is supported.
        if self.algorithm not in ('SAMME', 'SAMME.R'):
            raise ValueError("algorithm %s is not supported" % self.algorithm)

        # Check parameters.
        if self.learning_rate <= 0:
            raise ValueError("learning_rate must be greater than zero")

        if (self.base_estimator is None or
                isinstance(self.base_estimator, (BaseDecisionTree,
                                                 BaseForest))):
            DTYPE = np.float64  # from fast_dict.pxd
            dtype = DTYPE
            accept_sparse = 'csc'
        else:
            dtype = None
            accept_sparse = ['csr', 'csc']

        X, y = check_X_y(X, y, accept_sparse=accept_sparse, dtype=dtype,
                         y_numeric=is_regressor(self))
        
#         #DE-guided clustering
#         cluster_centers = compute_clusters(X,y)
#         cluster_stabilities = compute_cluster_stabilities(X,y,self.maj_class,cluster_centers)
#         #UNDERSAMPLING
#         boundary, nonboundary = compute_boundary_and_nonboundary_samples(X,y,self.maj_class,cluster_stabilities)
#         nonboundary_us = undersample_nonboundary_samples(nonboundary,cluster_stabilities)
#         X, y = undersampled_training_set(X,y,boundary,nonboundary_us,self.maj_class,self.min_class)


        declu = DECLUndersampling(H=self.H, alpha=self.us_alpha, CR=self.CR, F=self.F, POP_SIZE=self.POP_SIZE,
                                  NGEN=self.NGEN)
        X, y = declu.undersample(X,y)
        
        if sample_weight is None:
            # Initialize weights to 1 / n_samples.
            sample_weight = np.empty(X.shape[0], dtype=np.float64)
            sample_weight[:] = 1. / X.shape[0]
        else:
            sample_weight = check_array(sample_weight, ensure_2d=False)
            # Normalize existing weights.
            sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

            # Check that the sample weights sum is positive.
            if sample_weight.sum() <= 0:
                raise ValueError(
                    "Attempting to fit with a non-positive "
                    "weighted number of samples.")

        if minority_target is None:
            # Determine the minority class label.
            stats_c_ = Counter(y)
            maj_c_ = max(stats_c_, key=stats_c_.get)
            min_c_ = min(stats_c_, key=stats_c_.get)
            self.minority_target = min_c_
            self.majority_target = maj_c_
        else:
            self.minority_target = minority_target

        # Check parameters.
        self._validate_estimator()

        # Clear any previous fit results.
        self.estimators_ = []
        self.estimator_weights_ = np.zeros(self.n_estimators, dtype=np.float64)
        self.estimator_errors_ = np.ones(self.n_estimators, dtype=np.float64)

        random_state = check_random_state(self.random_state)
        
        for iboost in range(self.n_estimators):
            X_min = X[np.where(y == self.minority_target)]

            # SMOTE step.
            if len(X_min) >= self.smote_kneighbors:
                #create smote model
                n_maj = X[y==self.majority_target].shape[0]
                n_min = X_min.shape[0]
#                 self.smote = SMOTE(k_neighbors = self.smote_kneighbors,
#                                    sampling_strategy = {self.majority_target: n_maj,self.minority_target: n_min*2})
                self.smote = SMOTE(k_neighbors = self.smote_kneighbors)
                #fit and resample with smote
                X_res, y_res = self.smote.fit_resample(X, y)
#                 self.smote.fit(X_min)
#                 X_syn = self.smote.sample(n_samples = len(X_min))
                #select synthetic samples
                X_syn = X_res[y_res==self.minority_target][n_min:]
#                 y_syn = np.full(X_syn.shape[0], fill_value=self.minority_target,
#                                 dtype=np.int64)

                #DEguided selection of best synthetics
                self.desmote.fit(X,y,self.majority_target,self.minority_target,X_syn)
                selected_syn = []
                for i, value in enumerate(self.desmote.best_ind):
                    if self.desmote.best_ind[i]>0:
                        selected_syn.append(X_syn[i])
                selected_syn = np.array(selected_syn)
                y_syn = np.full(selected_syn.shape[0], fill_value=self.minority_target,
                                dtype=np.int64)
                
                # Normalize synthetic sample weights based on current training set.
                sample_weight_syn = np.empty(selected_syn.shape[0], dtype=np.float64)
                sample_weight_syn[:] = 1. / X.shape[0]
                
                # Combine the original and synthetic samples.
                print(" SE HAN INCLUIDO {} EJEMPLOS SINTÉTICOS".format(selected_syn.shape[0]))
                X_train = np.vstack((X, selected_syn))
                y_train = np.append(y, y_syn)
                
                # Combine the weights.
                sample_weight = \
                    np.append(sample_weight, sample_weight_syn).reshape(-1, 1)
                sample_weight = \
                    np.squeeze(normalize(sample_weight, axis=0, norm='l1'))

                # X, y, sample_weight = shuffle(X, y, sample_weight,
                #                              random_state=random_state)
                
            # Boosting step.
            sample_weight, estimator_weight, estimator_error = self._boost(
                iboost,
                X_train, y_train,
                sample_weight,
                random_state)

            # Early termination.
            if sample_weight is None:
                break

            self.estimator_weights_[iboost] = estimator_weight
            self.estimator_errors_[iboost] = estimator_error

            # Stop if error is zero.
            if estimator_error == 0:
                break
                
            sample_weight = sample_weight[:X.shape[0]]
            sample_weight_sum = np.sum(sample_weight)

            # Stop if the sum of sample weights has become non-positive.
            if sample_weight_sum <= 0:
                break

            if iboost < self.n_estimators - 1:
                # Normalize.
                sample_weight /= sample_weight_sum

        return self

In [33]:
def obtain_data(dataset_name):
    dataset = fetch_datasets()[dataset_name]
    return dataset.data,dataset.target

def convert_classes(y):
    default_classes = np.unique(y)
#     print("Default classes of the dataset were: ",default_classes)
    maj_class = -1
    min_class = 1
    if sum(y == default_classes[0]) > sum(y == default_classes[1]):
    #     maj_class = default_classes[0]
    #     min_class = default_classes[1]
        y[y==default_classes[0]] = maj_class
        y[y==default_classes[1]] = min_class
    else:
    #     maj_class = default_classes[1]
    #     min_class = default_classes[0]
        y[y==default_classes[1]] = maj_class
        y[y==default_classes[0]] = min_class

#     print("There are {} instances for the majoritary class".format(sum(y == maj_class)))
#     print("There are {} instanes for the minoritary class".format(sum(y == min_class)))
    return [maj_class,min_class], maj_class, min_class

def train(X_train, y_train, method_name, base_classifier, T):
    if method_name=='adaboost':
        clf = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=T)
    elif method_name=='RUSBoost':
        clf = RUSBoostClassifier(base_estimator=base_classifier,n_estimators=T,sampling_strategy='majority')
    elif method_name=='SMOTEBoost':
        clf = OversampleBoost(oversampling_algorithm='SMOTE',base_estimator=base_classifier, n_estimators=T)
    elif method_name=='SMOTETomekBoost':
        clf = OversampleBoost(oversampling_algorithm='SMOTE-TOMEK',base_estimator=base_classifier, n_estimators=T)
    elif method_name=='SMOTEENNBoost':
        clf = OversampleBoost(oversampling_algorithm='SMOTE-ENN',base_estimator=base_classifier, n_estimators=T)
    elif method_name=='DERSBoost':
        clf = DERSBoost(base_estimator=base_classifier, n_estimators=T, NGEN = 50)    
    start_time = time()
    clf.fit(X_train,y_train)
    elapsed_time = time() - start_time
    return clf,elapsed_time

def gmean_test(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    gmean = geometric_mean_score(y_test, y_pred)*100
    
    return gmean

def train_ensemble_method(dataset_name,method_name, T=10, k=5):
    #fetch data from dataset
    X, y = obtain_data(dataset_name)
    print("Dataset of size {}".format(X.shape))
    
    #convert, just in case, class labels to -1 (majoritary class) and 1 (minoritari class)
    classes, maj_class, min_class = convert_classes(y)
    
    #number of instances of each class and IR
    n_maj = X[y==maj_class].shape[0]
    n_min = X[y==min_class].shape[0]
    IR = n_maj/n_min
    print("There are {} instances for the majoritary class".format(n_maj))
    print("There are {} instanes for the minoritary class".format(n_min))
    print("IR of the dataset: ",IR)
    
    # Llamada al constructor del clasificador 
    dtc = DecisionTreeClassifier(criterion='entropy', max_depth=1)

    kf = StratifiedKFold(n_splits=k)

    gmean = []
    exec_time_mean = []
    for train_index, test_index in kf.split(X,y):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        clf,exec_time = train(X_train, y_train, method_name, dtc, T)
    
        partial_gmean = gmean_test(clf, X_test, y_test)
        
        gmean.append(partial_gmean)
        print("Gmean parcial: {}".format(partial_gmean))
        exec_time_mean.append(exec_time)
        
    print(gmean)
    rend = stats.mean(gmean)
    time = stats.mean(exec_time_mean)
    print("Rendimiento del clasificador {}: {}".format(method_name,rend))
    print("Tiempo medio de entrenamiento: {}".format(time))
    return rend, IR, time
    
    

In [9]:
imblearn_datasets = [
    'ecoli',
    'optical_digits',
    'satimage',
    'pen_digits',
    'abalone',
    'sick_euthyroid',
    'spectrometer',
    'car_eval_34',
    'isolet',
    'us_crime',
    'yeast_ml8',
    'scene',
    'libras_move',
    'thyroid_sick',
    'coil_2000',
    'arrhythmia',
    'solar_flare_m0',
    'oil',
    'car_eval_4',
    'wine_quality',
    'letter_img',
    'yeast_me2',
    'webpage',
    'ozone_level',
    'mammography',
    'protein_homo',
    'abalone_19'
]

In [36]:
r = train_ensemble_method('ecoli','DERSBoost')

Dataset of size (336, 7)
There are 301 instances for the majoritary class
There are 35 instanes for the minoritary class
IR of the dataset:  8.6


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Conjunto de entrenamiento original de tamaño: 268
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 240
 	 nº de ejemplos clase MINORITARIA: 28
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 139
Conjunto de entrenamiento actual de tamaño: 167


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


 SE HAN INCLUIDO 70 EJEMPLOS SINTÉTICOS


...

 SE HAN INCLUIDO 94 EJEMPLOS SINTÉTICOS


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


 SE HAN INCLUIDO 79 EJEMPLOS SINTÉTICOS


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


 SE HAN INCLUIDO 93 EJEMPLOS SINTÉTICOS
Gmean parcial: 84.51542547285166
[99.17694073609294, 63.80774695464942, 69.69320524371696, 57.32115042211109, 84.51542547285166]
Rendimiento del clasificador DERSBoost: 74.90289376588441
Tiempo medio de entrenamiento: 22.45224051475525


In [37]:
train_ensemble_method('spectrometer','DERSBoost')
train_ensemble_method('libras_move','DERSBoost')
train_ensemble_method('arrhythmia','DERSBoost')

Dataset of size (531, 93)
There are 486 instances for the majoritary class
There are 45 instanes for the minoritary class
IR of the dataset:  10.8


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Conjunto de entrenamiento original de tamaño: 424
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 388
 	 nº de ejemplos clase MINORITARIA: 36
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 256
Conjunto de entrenamiento actual de tamaño: 292


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


 SE HAN INCLUIDO 170 EJEMPLOS SINTÉTICOS


...

/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


 SE HAN INCLUIDO 166 EJEMPLOS SINTÉTICOS


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


 SE HAN INCLUIDO 142 EJEMPLOS SINTÉTICOS
Gmean parcial: 88.38418809879035
[76.55367474053136, 87.86882400805271, 62.11942956444276, 98.81652636251155, 88.38418809879035]
Rendimiento del clasificador DERSBoost: 82.74852855486574
Tiempo medio de entrenamiento: 592.1292934417725


(82.74852855486574, 17.08, 592.1292934417725)

In [75]:
rendimientos = []
IRlista = []
for dataset in imblearn_datasets:
    print(dataset.upper())
    r, IR = train_ensemble_method(dataset,'DERSBoost')
    rendimientos.append(r)
    IRlista.append(IR)
    print()


ECOLI
Dataset of size (336, 7)
There are 301 instances for the majoritary class
There are 35 instanes for the minoritary class
IR of the dataset:  8.6


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Conjunto de entrenamiento original de tamaño: 268
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 240
 	 nº de ejemplos clase MINORITARIA: 28
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 140
Conjunto de entrenamiento actual de tamaño: 168


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarn

Conjunto de entrenamiento original de tamaño: 269
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 241
 	 nº de ejemplos clase MINORITARIA: 28
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 140
Conjunto de entrenamiento actual de tamaño: 168


...

Conjunto de entrenamiento original de tamaño: 269
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 241
 	 nº de ejemplos clase MINORITARIA: 28
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 141
Conjunto de entrenamiento actual de tamaño: 169


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarn

[100.0, 100.0, 80.6225774829855, 67.25927091345493, 91.80725150319788]
Rendimiento del clasificador DERSBoost: 87.93781997992767

OPTICAL_DIGITS
Dataset of size (5620, 64)
There are 5066 instances for the majoritary class
There are 554 instanes for the minoritary class
IR of the dataset:  9.144404332129964


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


KeyboardInterrupt: 

No existe relación entre el IR ofrecido por el dataset y el rendimiento del clasificador

In [ ]:
print("La media de rendimiento entre todos los datasets es de :",stats.mean(rendimientos))

In [ ]:
print("n_samples default\nLa media de rendimiento entre todos los datasets es de :",stats.mean(rendimientos))

In [ ]:
print("n_samples n+*(floor(ir)-1)\nLa media de rendimiento entre todos los datasets es de :",stats.mean(rendimientos))

In [12]:
ders_datasets = ['ecoli','spectrometer','libras_move','arrhythmia']
dersRends = []
for dataset in ders_datasets:
    print(dataset.upper())
    r, IR = train_ensemble_method(dataset,'DERSBoost')
    dersRends.append(r)
    print()

ECOLI
Dataset of size (336, 7)
There are 301 instances for the majoritary class
There are 35 instanes for the minoritary class
IR of the dataset:  8.6
Conjunto de entrenamiento original de tamaño: 268
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 240
 	 nº de ejemplos clase MINORITARIA: 28
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 140
Conjunto de entrenamiento actual de tamaño: 168


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:31

Conjunto de entrenamiento original de tamaño: 269
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 241
 	 nº de ejemplos clase MINORITARIA: 28
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 141
Conjunto de entrenamiento actual de tamaño: 169


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarn

Conjunto de entrenamiento original de tamaño: 269
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 241
 	 nº de ejemplos clase MINORITARIA: 28
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 122
Conjunto de entrenamiento actual de tamaño: 150


...

Conjunto de entrenamiento original de tamaño: 362
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 342
 	 nº de ejemplos clase MINORITARIA: 20
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 312
Conjunto de entrenamiento actual de tamaño: 332


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarn

Conjunto de entrenamiento original de tamaño: 362
De los cuales:
 	 nº de ejemplos clase MAYORITARIA: 342
 	 nº de ejemplos clase MINORITARIA: 20
nº de ejemplos clase MAYORITARIA tras aplicar DE-guided UNDERSAMPLING: 284
Conjunto de entrenamiento actual de tamaño: 304


/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/cristina/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarn

[76.55367474053136, 88.92118276421004, 87.85013977691514, 87.85013977691514, 75.61512453751253]
Rendimiento del clasificador DERSBoost: 83.35805231921684



In [13]:
print(dersRends)

[84.14589029802556, 87.48296830905146, 83.34580562998774, 83.35805231921684]
